In [ ]:
using SerialPorts
using PyPlot
using FFTW

In [ ]:
#=================================================================
Receiving process
=================================================================#

receive_one = []
receive_two = []

# ser = SerialPort("COM3:", 9600)
ser = SerialPort("/dev/ttyACM0", 9600)

readavailable(ser)

#transmit and receive
write(ser, "c")

while bytesavailable(ser) < 1
    continue
end

sleep(0.05)
readavailable(ser) #removes the conversion complete line

#=================================================================#
# Get the first buffer
b = ""
readavailable(ser)

write(ser, "a") # Print DMA buffer
while bytesavailable(ser) < 1
    continue # wait for a response
end

while true
    if bytesavailable(ser) < 2
        sleep(0.005) # Wait and check again
        if bytesavailable(ser) < 1
            break
        end
    end

    b = string(b, readavailable(ser))

end

receive_one=split(b, ("\r\n"))

#=================================================================#
# Get the second buffer
b = ""
readavailable(ser)

write(ser, "b") # Print DMA buffer
while bytesavailable(ser) < 1
    continue # wait for a response
end

while true
    if bytesavailable(ser) < 2
        sleep(0.005) # Wait and check again
        if bytesavailable(ser) < 1
            break
        end
    end

    b = string(b, readavailable(ser))

end

close(ser)

receive_two=split(b, ("\r\n"))


println(length(receive_one))
println(length(receive_two))


In [ ]:
#=================================================================#
rc1 = []
i=1

while (i<length(receive_one)-1)
    push!(rc1,parse(Int,(receive_one[i])))
    i+=1
end

echo_one = (3.3/4096).*rc1

figure("Unprocessed Echo one")
plot(echo_one)

rc2 = []
i=1

while (i<length(receive_two)-1)
    push!(rc2,parse(Int,(receive_two[i])))
    i+=1
end

echo_2 = (3.3/4096).*rc2

figure("Unprocessed Echo two")
plot(echo_2)

EchoOne = (3.3/4096).*receive_one
EchoTwo= (3.3/4096).*receive_two

#end of receiving


In [ ]:

c = 343;          # Speed of sound in air in m/s
fs = 500000;       # This is the sample rate of the sonar.
dt = 1/fs;        # This is the sample spacing
r_max = 10;        # Maximum range in metres to which to simulate.
t_max = 2*r_max/c;

t = collect(0:dt:t_max);

# t = t[0:length(y)]
r = c*t/2;

f0 = 40000;       # Centre frequency is 40 kHz
B = 2000;         # Chirp bandwidth
T = 6E-3;         # Chirp pulse length
K = B/T;          # Chirp rate

td = 0.6*T;     #i'm very skeptical about this


In [ ]:

#"expected echoes"
v_tx1 = match[1:length(t)]
V_TX1 = fft(v_tx1);

v_tx2 = match[1:length(t)]
V_TX2 = fft(v_tx2);

#plots
figure("Expected Echo")
title("Expected Echo")
subplot(2,1,1)
plot(r,v_tx1)
subplot(2,1,2)
plot(r,v_tx2) #query

# received echos
v_rx1 = receive[1:length(t)]
V_RX1 = (fft(v_rx1));

v_rx2 = receive[1:length(t)]
V_RX2 = (fft(v_rx2));

#plots
figure("Unprocessed Received Echo")
title("Unprocessed Received Echo")
subplot(2,1,1)
plot(r,v_rx1)
subplot(2,1,2)
plot(r,v_rx2) #query
